In [1]:
import argparse
import utils
from pprint import pprint
import json

In [2]:
MYDICT = {'key': 'value'}

parser = argparse.ArgumentParser(description="")
parser.add_argument(
    "--config_retrieval",
    default="./config/retrieval_config.json",
    metavar="./config/retrieval_config.json", type=str, help=""
)
parser.add_argument(
    "--vectorizer_type",
    default="TfidfVectorizer",
    metavar="TfidfVectorizer", type=str, help=""
)
parser.add_argument('--vectorizer_parameters',
                    type=json.loads, default=MYDICT)
parser.add_argument(
    "--tokenizer_type",
    default="AutoTokenizer",
    metavar="AutoTokenizer", type=str, help=""
)
parser.add_argument(
    "--dataset_name", default = "./data/train_dataset",
    metavar="./data/train_dataset", type=str, help=""
)
parser.add_argument(
    "--model_name_or_path",
    default ="bert-base-multilingual-cased",
    metavar="bert-base-multilingual-cased",
    type=str,
    help="",
)
parser.add_argument("--data_path",default = "./data",
                    metavar="./data", type=str, help="")
parser.add_argument(
    "--context_path",
    default = "wikipedia_documents",
    metavar="wikipedia_documents", type=str, help=""
)
parser.add_argument(
    "--output_path",
    default="./retriever_result",
    metavar="./retriever_result", type=str, help=""
)

parser.add_argument("--use_faiss", default=False, metavar=False, type=bool, help="")

_StoreAction(option_strings=['--use_faiss'], dest='use_faiss', nargs=None, const=None, default=False, type=<class 'bool'>, choices=None, help='', metavar=False)

In [3]:
args = parser.parse_args([])
config = utils.read_json(args.config_retrieval)
parser.set_defaults(**config)
args = parser.parse_args([])

FileNotFoundError: [Errno 2] No such file or directory: './config/retrieval_config.json'

In [ ]:
pprint(vars(args))

## 1. arrow dataest analysis

In [ ]:
from datasets import Dataset, concatenate_datasets, load_from_disk

In [ ]:
# Test sparse
args.dataset_name = '../data/train_dataset'
org_dataset = load_from_disk(args.dataset_name)

In [ ]:
full_ds = concatenate_datasets(
    [
        org_dataset["train"].flatten_indices(),
        org_dataset["validation"].flatten_indices(),
    ]
)  # train dev 를 합친 4192 개 질문에 대해 모두 테스트
print("*" * 40, "query dataset", "*" * 40)
print(full_ds)

In [ ]:
print(full_ds)

In [ ]:
type(full_ds)

In [ ]:
print(len(full_ds["question"]))
print(len(full_ds["context"]))
print(len(full_ds["answers"]))

In [ ]:
for index, (q, c, a) in enumerate(zip(full_ds["question"], full_ds["context"], full_ds["answers"])):
    print(f'question : {q}')
    print(f'contenxt : {c}')
    print(f'answers : {a}')
    if index == 5:
        break
#     if len(a['text']) > 1:
#         print(f'question : {q}')
#         print(f'contenxt : {c}')
#         print(f'answers : {a}')

## 2. setting tokenizer

In [ ]:
import konlpy.tag
from transformers import AutoTokenizer
from importlib import import_module

In [ ]:
# args.tokenizer_type = "Mecab"

In [ ]:
if hasattr(import_module("transformers"), args.tokenizer_type):
    tokenizer_type = getattr(import_module("transformers"), args.tokenizer_type)
    tokenizer = tokenizer_type.from_pretrained(args.model_name_or_path, use_fast=False,)
    print(f'{args.tokenizer_type}')
elif hasattr(import_module("konlpy.tag"), args.tokenizer_type):
    tokenizer = getattr(import_module("konlpy.tag"), args.tokenizer_type)()
    print(f'{args.tokenizer_type}')
else:
    raise Exception(f"Use correct tokenizer type - {args.tokenizer_type}")

In [ ]:
print(type(tokenizer))

In [ ]:
tokenizer.tokenize('일등이 아니어도 괜찮아')

In [ ]:
tokenizer2 = getattr(import_module("konlpy.tag"), "Mecab")()

In [ ]:
tokenizer2.morphs('일등이 아니어도 괜찮아')

## 3. Setting Sparse Retrieval

In [ ]:
print(args.vectorizer_type)
print(args.vectorizer_parameters)

In [ ]:
# retriever = SparseRetrieval(
#     retrieval_type = args.retrieval_type,
#     retrieval_parameters = args.retrieval_parameters,
#     tokenize_fn=tokenizer.tokenize if args.tokenizer_type == "AutoTokenizer" else tokenizer.morphs,
#     data_path=args.data_path,
#     context_path=args.context_path,
# )

In [ ]:
from typing import List, NoReturn, Optional, Tuple, Union
import faiss
import numpy as np
import pandas as pd
import os

In [ ]:
data_path = "../data/wikipedia_documents.json"
with open(data_path, "r", encoding="utf-8") as f:
    wiki = json.load(f)

In [ ]:
wiki['0']

In [ ]:
k = [v["text"] for v in wiki.values()]

In [ ]:
# 순서대로 중복 제거
# ('key1', 'key2', 'key3', 'key1', 'key4', 'key2') -> ['key1', 'key2', 'key3', 'key4']
contexts = list(
    dict.fromkeys([v["text"] for v in wiki.values()])
) 

In [ ]:
len(contexts)

## 4. setting retrieval_type 

In [ ]:
import retriever
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# k = retriever.BM25()

In [ ]:
print(f'args.vectorizer_type : {args.vectorizer_type}\n')
print(f'args.vectorizer_parameters : {args.vectorizer_parameters}\n')
print(f'tokenizer : {tokenizer}')
print(f'tokenizer.tokenize : {tokenizer.tokenize}')


In [ ]:
# Transform by vectorizer
# tfidfv = TfidfVectorizer(
#     tokenizer=tokenize_fn, **args.retrieval_parameters
# )
# print(tfidfv)

In [ ]:
# print(hasattr(import_module("sklearn.feature_extraction.text"), args.vectorizer_type))

In [ ]:
# print(hasattr(import_module("retriever"), "BM25"))

In [ ]:
tokenize_fn_1 = tokenizer.tokenize
print(tokenize_fn_1, '\n')

from konlpy.tag import Mecab
tokenizer2 = Mecab()
tokenize_fn_2 =tokenizer2.morphs
print(tokenize_fn_2)

In [ ]:
args.vectorizer_type = "TfidfVectorizer"
tokenize_fn = tokenize_fn_1

if hasattr(import_module("sklearn.feature_extraction.text"), args.vectorizer_type):
    vectorizer_type = getattr(import_module("sklearn.feature_extraction.text"), args.vectorizer_type)
    vectorizer = vectorizer_type(tokenizer=tokenize_fn, **args.vectorizer_parameters)
    print(f'{vectorizer}')
    
elif hasattr(import_module("retriever"), args.vectorizer_type):
    vectorizer_type = getattr(import_module("retriever"), args.vectorizer_type)
    vectorizer = vectorizer_type(tokenize_fn, args.vectorizer_parameters)
    print(f'{vectorizer}')
    
else:
    raise Exception(f"Use correct tokenizer type : Current tokenizer : {args.vectorizer_type}")

In [ ]:
args.vectorizer_type = "TfidfVectorizer"
tokenize_fn = tokenize_fn_2

if hasattr(import_module("sklearn.feature_extraction.text"), args.vectorizer_type):
    vectorizer_type = getattr(import_module("sklearn.feature_extraction.text"), args.vectorizer_type)
    vectorizer = vectorizer_type(tokenizer=tokenize_fn, **args.vectorizer_parameters)
    print(f'{vectorizer}')
    
elif hasattr(import_module("retriever"), args.vectorizer_type):
    vectorizer_type = getattr(import_module("retriever"), args.vectorizer_type)
    print(f'tokenize_fn : {tokenize_fn}')
    print(f'vectorizer_type : {vectorizer_type}')
    vectorizer = vectorizer_type(tokenize_fn, args.vectorizer_parameters)
    print(f'{vectorizer.vectorizer}')
    
else:
    raise Exception(f"Use correct tokenizer type : Current tokenizer : {args.vectorizer_type}")

In [ ]:
args.vectorizer_type = "BM25"
tokenize_fn = tokenize_fn_2

if hasattr(import_module("sklearn.feature_extraction.text"), args.vectorizer_type):
    vectorizer_type = getattr(import_module("sklearn.feature_extraction.text"), args.vectorizer_type)
    vectorizer = vectorizer_type(tokenizer=tokenize_fn, **args.vectorizer_parameters)
    print(f'{vectorizer}')
    
elif hasattr(import_module("retriever"), args.vectorizer_type):
    vectorizer_type = getattr(import_module("retriever"), args.vectorizer_type)
    print(f'tokenize_fn : {tokenize_fn}')
    print(f'vectorizer_type : {vectorizer_type}')
    vectorizer = vectorizer_type(tokenize_fn, args.vectorizer_parameters)
    print(f'{vectorizer.vectorizer}')
    
else:
    raise Exception(f"Use correct tokenizer type : Current tokenizer : {args.vectorizer_type}")

In [ ]:
args.vectorizer_type = "BM25"
tokenize_fn = tokenize_fn_1

if hasattr(import_module("sklearn.feature_extraction.text"), args.vectorizer_type):
    vectorizer_type = getattr(import_module("sklearn.feature_extraction.text"), args.vectorizer_type)
    vectorizer = vectorizer_type(tokenizer=tokenize_fn, **args.vectorizer_parameters)
    print(f'{vectorizer}')
    
elif hasattr(import_module("retriever"), args.vectorizer_type):
    vectorizer_type = getattr(import_module("retriever"), args.vectorizer_type)
    print(f'tokenize_fn : {tokenize_fn}')
    print(f'vectorizer_type : {vectorizer_type}')
    vectorizer = vectorizer_type(tokenize_fn, args.vectorizer_parameters)
    print(f'{vectorizer.vectorizer}')
    
else:
    raise Exception(f"Use correct tokenizer type : Current tokenizer : {args.vectorizer_type}")

In [ ]:
args.vectorizer_type = "BM25"
tokenize_fn = tokenize_fn_2

if hasattr(import_module("sklearn.feature_extraction.text"), args.vectorizer_type):
    vectorizer_type = getattr(import_module("sklearn.feature_extraction.text"), args.vectorizer_type)
    vectorizer = vectorizer_type(tokenizer=tokenize_fn, **args.vectorizer_parameters)
    print(f'{vectorizer}')
    
elif hasattr(import_module("retriever"), args.vectorizer_type):
    vectorizer_type = getattr(import_module("retriever"), args.vectorizer_type)
    print(f'tokenize_fn : {tokenize_fn}')
    print(f'vectorizer_type : {vectorizer_type}')
    vectorizer = vectorizer_type(tokenize_fn, args.vectorizer_parameters)
    print(f'{vectorizer.vectorizer}')
    
else:
    raise Exception(f"Use correct tokenizer type : Current tokenizer : {args.vectorizer_type}")

In [ ]:
if hasattr(import_module("sklearn.feature_extraction.text"), args.vectorizer_type):
    vectorizer_type = getattr(import_module("sklearn.feature_extraction.text"), args.vectorizer_type)
    vectorizer = vectorizer_type(tokenizer=tokenize_fn, **args.vectorizer_parameters)
    print(f'{vectorizer}')
    
elif hasattr(import_module("retriever"), args.vectorizer_type):
    vectorizer_type = getattr(import_module("retriever"), args.vectorizer_type)
    vectorizer = vectorizer_type(tokenize_fn, args.vectorizer_parameters)
    print(f'{vectorizer.vectorizer}')
    
else:
    raise Exception(f"Use correct tokenizer type : Current tokenizer : {args.vectorizer_type}")

### 4. setting retrieval result directory with incrementing 

In [ ]:
pprint(vars(args))

In [ ]:
type(args.output_path)

In [ ]:
if args.tokenizer_type == "AutoTokenizer":
    output_path = args.output_path + f'/{args.vectorizer_type}_{args.model_name_or_path}_{args.context_path}'
else:
    output_path = args.output_path + f'/{args.vectorizer_type}_{args.tokenizer_type}_{args.context_path}'

In [ ]:
print(output_path)

In [ ]:
# i = 1
# while True:
#     path = f'{output_path}_{i}/'
#     print(path)
#     if not os.path.exists(path):
#         os.makedirs(os.path.dirname(path), exist_ok=True)
#         break
#     i += 1


In [ ]:
print(output_path)

In [ ]:
import utils
output_path = utils.increment_directory(output_path)


In [ ]:
print(output_path)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
if args.tokenizer_type == "AutoTokenizer":
    output_path = args.output_path + f'/{args.vectorizer_type}_{args.model_name_or_path}_{args.context_path}'
else:
    output_path = args.output_path + f'/{args.vectorizer_type}_{args.tokenizer_type}_{args.context_path}'

In [ ]:
output_path